<a href="https://colab.research.google.com/github/Homura-san/pyspark/blob/main/Pyspark_ElasticSearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Instalando o Pyspark**

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [ ]:
# Fazendo download
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz

# Descompactando os arquivos
!tar xf spark-3.1.2-bin-hadoop2.7.tgz

tar: spark-3.1.2-bin-hadoop2.7.tgz: Cannot open: No such file or directory
tar: Error is not recoverable: exiting now


In [ ]:
!tar xf spark-3.1.2-bin-hadoop2.7.tgz

In [ ]:
import os

# Definindo a variável de ambiente do Java
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

# Definindo a variável de ambiente do Spark
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

In [ ]:
# instalando a findspark
!pip install -q findspark

In [ ]:
# Importando a findspark
import findspark

# Iniciando o findspark
findspark.init()

In [ ]:
# importando o pacote necessário para iniciar uma seção Spark
from pyspark.sql import SparkSession

# iniciando o spark context
sc = SparkSession.builder.master('local[*]').getOrCreate()

# Verificando se a sessão foi criada
sc

# **Instalando o ElasticSearch**

In [ ]:
!pip install tensorflow-io
!pip install elasticsearch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.0/28.0 MB 47.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 387.9/387.9 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 7.8 MB/s eta 0:00:00


In [ ]:
import os
import time
from sklearn.model_selection import train_test_split
from elasticsearch import Elasticsearch
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
import tensorflow_io as tfio

### Validando imports do tf e tfio

In [ ]:
print("tensorflow-io version: {}".format(tfio.__version__))
print("tensorflow version: {}".format(tf.__version__))

tensorflow-io version: 0.32.0
tensorflow version: 2.12.0


## Baixando e instalando a instância do Elasticsearch

In [ ]:
%%bash

wget -q https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-oss-7.9.2-linux-x86_64.tar.gz
wget -q https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-oss-7.9.2-linux-x86_64.tar.gz.sha512
tar -xzf elasticsearch-oss-7.9.2-linux-x86_64.tar.gz
sudo chown -R daemon:daemon elasticsearch-7.9.2/
shasum -a 512 -c elasticsearch-oss-7.9.2-linux-x86_64.tar.gz.sha512 

elasticsearch-oss-7.9.2-linux-x86_64.tar.gz: OK


In [ ]:
%%bash --bg

sudo -H -u daemon elasticsearch-7.9.2/bin/elasticsearch

In [ ]:
# Sleep for few seconds to let the instance start.
time.sleep(20)

In [ ]:
%%bash

ps -ef | grep elasticsearch

root         700     698  0 12:58 ?        00:00:00 sudo -H -u daemon elasticsearch-7.9.2/bin/elasticsearch
daemon       701     700 28 12:58 ?        00:00:21 /content/elasticsearch-7.9.2/jdk/bin/java -Xshare:auto -Des.networkaddress.cache.ttl=60 -Des.networkaddress.cache.negative.ttl=10 -XX:+AlwaysPreTouch -Xss1m -Djava.awt.headless=true -Dfile.encoding=UTF-8 -Djna.nosys=true -XX:-OmitStackTraceInFastThrow -XX:+ShowCodeDetailsInExceptionMessages -Dio.netty.noUnsafe=true -Dio.netty.noKeySetOptimization=true -Dio.netty.recycler.maxCapacityPerThread=0 -Dio.netty.allocator.numDirectArenas=0 -Dlog4j.shutdownHookEnabled=false -Dlog4j2.disable.jmx=true -Djava.locale.providers=SPI,COMPAT -Xms1g -Xmx1g -XX:+UseG1GC -XX:G1ReservePercent=25 -XX:InitiatingHeapOccupancyPercent=30 -Djava.io.tmpdir=/tmp/elasticsearch-14549110779989349442 -XX:+HeapDumpOnOutOfMemoryError -XX:HeapDumpPath=data -XX:ErrorFile=logs/hs_err_pid%p.log -Xlog:gc*,gc+age=trace,safepoint:file=logs/gc.log:utctime,pid,tags:fileco

In [ ]:
%%bash

curl -sX GET "localhost:9200/"

{
  "name" : "34dc521b6ff4",
  "cluster_name" : "elasticsearch",
  "cluster_uuid" : "sgEtJSgmQAWU2hFhvYbtwg",
  "version" : {
    "number" : "7.9.2",
    "build_flavor" : "oss",
    "build_type" : "tar",
    "build_hash" : "d34da0ea4a966c4e49417f2da2f244e3e97b4e6e",
    "build_date" : "2020-09-23T00:45:33.626720Z",
    "build_snapshot" : false,
    "lucene_version" : "8.6.2",
    "minimum_wire_compatibility_version" : "6.8.0",
    "minimum_index_compatibility_version" : "6.0.0-beta1"
  },
  "tagline" : "You Know, for Search"
}


# **Importando Arquivo para teste**

#### * Via Pyspark

In [ ]:
# Fazendo download do arquivo
!wget --verbose --show-progress --no-check-certificate https://raw.githubusercontent.com/jonates/opendata/master/receita_federal/receita_federal_arrecadacao_por_UF_2020.csv

--2023-05-18 12:57:37--  https://raw.githubusercontent.com/jonates/opendata/master/receita_federal/receita_federal_arrecadacao_por_UF_2020.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6216 (6.1K) [text/plain]
Saving to: ‘receita_federal_arrecadacao_por_UF_2020.csv’

receita_federal_arr 100%[===================>]   6.07K  --.-KB/s    in 0s      

2023-05-18 12:57:37 (61.4 MB/s) - ‘receita_federal_arrecadacao_por_UF_2020.csv’ saved [6216/6216]



In [ ]:
# carregando um conjunto de dados que baixamos da internet
receitafederal = sc.read.csv(
    path = "/content/receita_federal_arrecadacao_por_UF_2020.csv", 
    inferSchema = True, 
    header = True,
    sep = ';', 
    encoding = "UTF-8")

In [ ]:
# Verificando o tipo de objeto criado
type(receitafederal)

pyspark.sql.dataframe.DataFrame

In [ ]:
# Espiando o dataset
receitafederal.show()

+---+------------+----+------------------------+------------------------+-------------+---------------------------+-------------+--------------+-------------------------------+-------------------------------+-------------------------+--------------+-----------------------------+--------------+-----------------+------------------------------------------------------+-----------------------------+
| uf|      regiao| ano|imposto_sobre_importacao|imposto_sobre_exportacao|    ipi_total|imposto_sobre_a_renda_total|         irpf|          irpj|imposto_s_renda_retido_na_fonte|imposto_s_operacoes_financeiras|imposto_territorial_rural|        cofins|contribuicao_para_o_pis_pasep|          csll|cide_combustiveis|cpsss_contrib_p_o_plano_de_segurid_social_serv_publico|outras_receitas_administradas|
+---+------------+----+------------------------+------------------------+-------------+---------------------------+-------------+--------------+-------------------------------+----------------------------

In [ ]:
# Verificando o schema() deste sparkdataframe
receitafederal.printSchema()

root
 |-- uf: string (nullable = true)
 |-- regiao: string (nullable = true)
 |-- ano: integer (nullable = true)
 |-- imposto_sobre_importacao: string (nullable = true)
 |-- imposto_sobre_exportacao: string (nullable = true)
 |-- ipi_total: string (nullable = true)
 |-- imposto_sobre_a_renda_total: string (nullable = true)
 |-- irpf: string (nullable = true)
 |-- irpj: string (nullable = true)
 |-- imposto_s_renda_retido_na_fonte: string (nullable = true)
 |-- imposto_s_operacoes_financeiras: string (nullable = true)
 |-- imposto_territorial_rural: string (nullable = true)
 |-- cofins: string (nullable = true)
 |-- contribuicao_para_o_pis_pasep: string (nullable = true)
 |-- csll: string (nullable = true)
 |-- cide_combustiveis: string (nullable = true)
 |-- cpsss_contrib_p_o_plano_de_segurid_social_serv_publico: string (nullable = true)
 |-- outras_receitas_administradas: string (nullable = true)



In [ ]:
# importando os métodos com funções para transformações de variáveis
from pyspark.sql.functions import *


In [ ]:
# Transformando o atributo irpf em numerica
receitafederal = receitafederal.withColumn(
    colName = 'irpf', 
    col = regexp_replace('irpf',',','.').cast('float')
    )

# Inspecionando o resultado
receitafederal.select('irpf').printSchema()

root
 |-- irpf: float (nullable = true)



In [ ]:
# Verificando o total do irpf por Região do Brasil
receitafederal.groupBy('regiao').sum('irpf').orderBy('regiao').show()

+------------+--------------+
|      regiao|     sum(irpf)|
+------------+--------------+
|Centro-Oeste| 3.354157696E9|
|    Nordeste| 4.303029696E9|
|       Norte| 1.404179308E9|
|     Sudeste|2.496098528E10|
|         Sul| 7.380957184E9|
|       Total|4.140331008E10|
+------------+--------------+



### * Via ElasticSearch

In [ ]:
dataset_url = 'http://storage.googleapis.com/download.tensorflow.org/data/petfinder-mini.zip'
csv_file = 'datasets/petfinder-mini/petfinder-mini.csv'
tf.keras.utils.get_file('petfinder_mini.zip', dataset_url,
                        extract=True, cache_dir='.')
pf_df = pd.read_csv(csv_file)

In [ ]:
pf_df.head()

,Type,Age,Breed1,Gender,Color1,Color2,MaturitySize,FurLength,Vaccinated,Sterilized,Health,Fee,Description,PhotoAmt,AdoptionSpeed
0,Cat,3,Tabby,Male,Black,White,Small,Short,No,No,Healthy,100,Nibble is a 3+ month old ball of cuteness. He ...,1,2
1,Cat,1,Domestic Medium Hair,Male,Black,Brown,Medium,Medium,Not Sure,Not Sure,Healthy,0,I just found it alone yesterday near my apartm...,2,0
2,Dog,1,Mixed Breed,Male,Brown,White,Medium,Medium,Yes,No,Healthy,0,Their pregnant mother was dumped by her irresp...,7,3
3,Dog,4,Mixed Breed,Female,Black,Brown,Medium,Short,Yes,No,Healthy,150,"Good guard dog, very alert, active, obedience ...",8,2
4,Dog,1,Mixed Breed,Male,Black,No Color,Medium,Short,No,No,Healthy,0,This handsome yet cute boy is up for adoption....,3,2


In [ ]:
# In the original dataset "4" indicates the pet was not adopted.
pf_df['target'] = np.where(pf_df['AdoptionSpeed']==4, 0, 1)

# Drop un-used columns.
pf_df = pf_df.drop(columns=['AdoptionSpeed', 'Description'])

In [ ]:
# Number of datapoints and columns
len(pf_df), len(pf_df.columns)

(11537, 14)

In [ ]:
train_df, test_df = train_test_split(pf_df, test_size=0.3, shuffle=True)
print("Number of training samples: ",len(train_df))
print("Number of testing sample: ",len(test_df))

Number of training samples:  8075
Number of testing sample:  3462


# Usando arquivo prórpio

In [ ]:
# Baixando arquivo csv
!wget --verbose --show-progress --no-check-certificate https://raw.githubusercontent.com/Homura-san/pyspark/main/IPCA%20MESES%202022.csv

- Via pyspark

In [ ]:
inflacao = sc.read.csv(
    path = "/content/IPCA MESES 2022.csv", 
    inferSchema = True, 
    header = True,
    sep = ';', 
    encoding = "UTF-8")

In [ ]:
# Verificando o tipo de objeto criado
type(inflacao)

pyspark.sql.dataframe.DataFrame

In [ ]:
# Olhando o dataset
inflacao.show()

+-----------------+-----------+-----------------------+-------------------+
|MES DE REFERENCIA|IPCA NO MES|IPCA ACUMULADO 12 MESES|IPCA ACUMULADO 2022|
+-----------------+-----------+-----------------------+-------------------+
|              Jan|      0,54%|                   null|              0,54%|
|              Fev|      1,01%|                   null|              1,56%|
|              Mar|      1,62%|                   null|              3,20%|
|              Abr|      1,06%|                   null|              4,29%|
|              Mai|      0,47%|                   null|              4,78%|
|              Jun|      0,67%|                   null|              5,49%|
|              Jul|     -0,68%|                   null|              4,77%|
|              Ago|     -0,36%|                   null|              4,39%|
|              Set|     -0,29%|                   null|              4,09%|
|              Out|      0,59%|                   null|              4,70%|
|           

- Via ElasticSearch

In [ ]:
# Baixando arquivo csv
!wget --verbose --show-progress --no-check-certificate https://raw.githubusercontent.com/Homura-san/pyspark/main/IPCA.csv

csv_f = "/content/IPCA.csv"

ipca = pd.read_csv(csv_f)

--2023-05-18 14:18:39--  https://raw.githubusercontent.com/Homura-san/pyspark/main/IPCA.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 361 [text/plain]
Saving to: ‘IPCA.csv’

IPCA.csv            100%[===================>]     361  --.-KB/s    in 0s      

2023-05-18 14:18:39 (14.1 MB/s) - ‘IPCA.csv’ saved [361/361]



In [ ]:
ipca.head()

,MES DE REFERENCIA,IPCA NO MES,IPCA ACUMULADO 12 MESES,IPCA ACUMULADO 2022
0,Jan,0.54%,10.38%,0.54%
1,Fev,1.01%,10.54%,1.56%
2,Mar,1.62%,11.30%,3.20%
3,Abr,1.06%,12.13%,4.29%
4,Mai,0.47%,11.73%,4.78%
